In [1]:
from MicrogridPSO_module import *
from MicrogridPSO_module_flowchart import *
from MicrogridPSO_module_PSO import *
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# inspect function
#print(inspect.getmembers(MicrogridPSO_initialize, inspect.isfunction))

# inspect methods
#print(inspect.getmembers(MicrogridPSO_initialize, inspect.ismethod))

# inspect object & methods
# print(dir(MicrogridPSO_initialize))

In [ ]:
# How to initialize
# Class check
# CSVを読み込みインスタンスを作成。
PSO = MicrogridPSO_initialize("Target_input.csv")

# 初期値を設定
PSO.set_initial_input_values({"number_demand": 1,
                              "pv_capacity_per_unit": 245,
                              "wind_capacity_per_unit": 2.3,
                              "SOC_max[%]": 0.8,
                              "SOC_min[%]": 0.2,
                              "SOC_start[%]": 0.5
                              })

# 初期値を設定
PSO.set_initial_cost_parameters({"It_PV_1kW[yen/year]": [726383.3333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 "Mt_PV_1kW[yen/year]": [4329] * 20,
                                 "Ft_PV_1kW[yen/year]": [0]*20,
                                 "It_Wind_1kW[yen/year]": [302274.7826, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 "Mt_Wind_1kW[yen/year]": [662.785213] * 20,
                                 "Ft_Wind_1kW[yen/year]": [0]*20,
                                 "It_Diesel_1kW[yen/year]": [33611.11, 33611.11, 0, 33611.11, 33611.11, 0, 33611.11, 33611.11, 0, 33611.11,
                                                             33611.11, 0, 33611.11, 33611.11, 0, 33611.11, 33611.11, 0, 33611.11, 33611.11],
                                 "Mt_Diesel_1kW[yen/year]": [8.8] * 20,
                                 "Diesel_Pf": 129,
                                 "Diesel_Adg": 0.2461,
                                 "Diesel_Bdg": 0.081451,
                                 "It_Battery_1kW[yen/year]": [13540, 0, 0, 0, 13540, 0, 0, 0, 13540, 0, 0, 0, 13540, 0, 0, 0, 13540, 0, 0, 0],
                                 "Mt_Battery_1kW[yen/year]": [1100] * 20,
                                 "Ft_Battery_1kW[yen/year]": [0]*20,
                                 "Sell_income_from_trashed[kWh/yen]": [0]*20,
                                 "r[yen/year]": [0.0234375]*20,
                                 "operation_year": list(range(1, 21))
                                 })

# PSOの中身を確認
# print(dir(PSO))
# print(vars(PSO))
# len(PSO.Target_input.index)

In [ ]:
%%time

#計算結果を実行
iterations_PSO(PSO)

Inform the number of iterations: 7
Inform the number of particles: 8
Inform w: 0.7
Inform c1: 0.5
Inform c2: 0.6
{'pv_cap_max': 0, 'wind_cap_max': 0, 'battery_cap_max': 0, 'diesel_max': 0} 
iterations: 7 n_particles: 8 w: 0.7 c1: 0.5 c2: 0.6 particle: {'particle_position_vector': array([[7.53929441e+02, 9.68866167e+00, 5.80343223e+00, 3.43665078e+00],
       [1.11778975e+03, 3.47127945e+00, 1.38750943e+01, 4.08068882e+00],
       [2.48783497e+03, 4.40830447e+00, 1.65525424e+01, 1.17144234e+00],
       [1.76209363e+03, 8.23669592e+00, 4.50749229e+00, 1.53188116e+00],
       [5.17256110e+02, 8.62443630e+00, 1.49899928e+01, 1.01129357e+01],
       [1.63409601e+03, 4.34036171e+00, 1.08189531e+01, 4.79020348e+00],
       [1.68147303e+03, 8.97246206e+00, 2.12960944e+01, 9.55864133e+00],
       [3.12618460e+02, 3.46924325e+00, 8.50127005e+00, 4.27728217e+00]]), 'personal_best_position': array([[7.53929441e+02, 9.68866167e+00, 5.80343223e+00, 3.43665078e+00],
       [1.11778975e+03, 3.47127945

In [ ]:
# print(PSO.best['gbest_fitness_value'])
df=PSO.best['table']
df.to_csv('Result/PSO_result_no_income.csv',encoding="SHIFT-JIS")  
df

In [ ]:
best_cost_list = np.array(PSO.best_cost_list)
gbest_list = np.array(PSO.global_best_list)
iteration_list = np.array(PSO.iteration_list)
plot_list = pd.DataFrame({'iteration': np.array(PSO.iteration_list)+1,
                          'cost': np.array(PSO.best_cost_list)})
print(max(best_cost_list), min(best_cost_list))
# plot_list

In [ ]:
plot_list

In [ ]:
plot_list.plot(x="iteration", y="cost", title='Total cost in each iteration')
plt.savefig('Result/Total cost in each iteration.png')

In [ ]:
# 計算結果テキストファイルの作成
f = open('Result/PSO_result_parameters.txt', 'w')  # 書き込みモードで開く
f_content = 'LCOE='+str(np.sum(PSO.best['SCL'])/np.sum(PSO.best['SEL']))+' global_best_position: '+str(PSO.best['global_best_position'])+' global_best_fitness_value: '+str(PSO.best['global_best_fitness_value'])+str(PSO.best["variables"]) + \
    str(PSO.initial_cost_parameters)+'SCL:'+str(PSO.best['SCL'])+'SEL:'+str(PSO.best['SEL'])
f.write(f_content)  # 引数の文字列をファイルに書き込む
f.close()  # ファイルを閉じる

In [ ]:
PSO.best['SCL']

In [ ]:
PSO.best['SEL']

In [ ]:
dir(PSO)